<a href="https://colab.research.google.com/github/AziizBg/retriever/blob/synthetic_data/Synthetic_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AziizBg/retriever.git

Cloning into 'retriever'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 174 (delta 32), reused 155 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (174/174), 8.16 MiB | 15.83 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [14]:
!git pull

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 1.26 KiB | 647.00 KiB/s, done.
From https://github.com/AziizBg/retriever
   755b506..db26b82  synthetic_data -> origin/synthetic_data
Updating 755b506..db26b82
Fast-forward
 nvidia_api.py | 89 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 89 insertions(+)
 create mode 100644 nvidia_api.py


In [1]:
%cd retriever

/content/retriever


In [3]:
!pip install -r requirements.txt

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.4/500.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 78.7

In [2]:
!git checkout synthetic_data

Branch 'synthetic_data' set up to track remote branch 'synthetic_data' from 'origin'.
Switched to a new branch 'synthetic_data'


In [18]:
import os
import json
import glob
from pathlib import Path
import requests
import time
from typing import List, Dict, Any
import logging
from dotenv import load_dotenv
from openai import OpenAI
from retriever.nvidia_api import call_nvidia_api, get_nvidia_client


In [19]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [56]:
def load_json_files(base_dir: str) -> List[Dict[str, Any]]:
    """Load all JSON files from the given directory and its subdirectories."""
    json_files = []
    for json_file in glob.glob(os.path.join(base_dir, "*.json")):
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                json_files.append({
                    'file_path': json_file,
                    'content': data
                })
            logger.info(f"Successfully loaded {json_file}")
        except Exception as e:
            logger.error(f"Error loading {json_file}: {str(e)}")
    return json_files

In [32]:
# Get the client
client = get_nvidia_client()

# Example prompts
system_prompt = "You are a helpful assistant."
user_prompt = "Hello, how are you?"

# Make the API call
response = call_nvidia_api(client, system_prompt, user_prompt)
print("API Response:", response)

API Response: Hello! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. It's great to meet you!

How can I help you today? Do you have something specific in mind, or would you like me to suggest some ways I can assist you? I can help with things like:

* Answering questions on a wide range of topics
* Generating text or ideas
* Translation
* Summarizing content
* Offering suggestions or advice
* Chatting for fun!

Let me know how I can make your day a little easier or more interesting!


In [ ]:
def generate_qa_pairs(client, content: Dict[str, Any]) -> List[Dict[str, str]]:
    """Generate Q&A pairs using NVIDIA's LLM API."""

    # System prompt to set the context and behavior
    system_prompt = """You are a medical expert tasked with generating high-quality question-answer pairs from medical information.
    Your responses should be:
    1. Medically accurate and evidence-based
    2. Clear and concise
    3. Appropriate for both medical professionals and patients
    4. Focused on key medical concepts, treatments, and implications
    5. Free from any harmful or misleading information

    Format your response as a JSON array of objects with 'question' and 'answer' fields."""

    # User prompt to guide the specific generation task
    user_prompt = f"""Based on the following medical information, generate relevant question-answer pairs.
    Make the questions specific and the answers detailed but concise.

    Information:
    {json.dumps(content, indent=2)}

    Return ONLY a JSON array of objects with 'question' and 'answer' fields, with no additional text or markdown formatting.
    Example format:
    [
        {{
            "question": "What causes tuberculosis?",
            "answer": "Tuberculosis is caused by the bacterium Mycobacterium tuberculosis."
        }}
    ]"""

    try:
        # Make the API call with both prompts
        response = call_nvidia_api(client, system_prompt, user_prompt)

        # print("response")
        # print(response)

        response = response.strip()
        if "```json" in response:
            response = response.split("```json")[1]
        if "```" in response:
            response = response.split("```")[0]
        response = response.strip()

        # print("modified response")
        # print(response)


        # Parse the response as JSON
        try:
            qa_pairs = json.loads(response)
            if not isinstance(qa_pairs, list):
                raise ValueError("Response is not a list")
            for pair in qa_pairs:
                if not isinstance(pair, dict) or 'question' not in pair or 'answer' not in pair:
                    raise ValueError("Invalid Q&A pair format")

            print("qa_pairs")
            print(qa_pairs)
            return qa_pairs
        except json.JSONDecodeError as e:
            logger.error(f"Error parsing JSON response: {str(e)}")
            return []

    except Exception as e:
        logger.error(f"Error generating Q&A pairs: {str(e)}")
        return []


In [ ]:
raw_data_dir = '/content/retriever/raw_files/tuberculosis'
all_data = []

for filename in glob.glob(os.path.join(raw_data_dir, '*.json')):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_data.append(data)
        logger.info(f"Successfully loaded {filename}")
    except Exception as e:
        logger.error(f"Error loading {filename}: {str(e)}")

output_file = '/content/retriever/raw_files/all_tuberculosis_data.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_data, f, indent=2)

logger.info(f"Merged all JSON files into {output_file}")
print(f"Merged all JSON files into {output_file}")


In [57]:
def main():
    base_dir = os.path.join(os.getcwd(), "raw_files")
    output_dir = os.path.join(os.getcwd(), "generated_qa")
    os.makedirs(output_dir, exist_ok=True)

    # Get the client
    client = get_nvidia_client()

    # Load all JSON files
    json_files = load_json_files(base_dir)
    print(f"Found {len(json_files)} JSON files")

    # Process each file
    for file_data in json_files:
        file_path = file_data['file_path']
        content = file_data['content']

        # Generate Q&A pairs
        qa_pairs = generate_qa_pairs(client, content)

        if qa_pairs:
            # Create output filename
            relative_path = os.path.relpath(file_path, base_dir)
            output_filename = os.path.join(output_dir, f"{Path(relative_path).stem}_qa.json")

            # Save Q&A pairs
            with open(output_filename, 'w', encoding='utf-8') as f:
                json.dump(qa_pairs, f, indent=2)
            print(f"Saved Q&A pairs to {output_filename}")

        # Add a small delay to avoid rate limiting
        time.sleep(20)

if __name__ == "__main__":
    main()

Found 5 JSON files


KeyboardInterrupt: 